<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/02_Envio_parceiro_magalu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install workadays # pacote para calculo data prometida e data limite postagem
import pandas as pd
import re
import datetime
from workadays import workdays as wd
# Bibliotecas Google Golab
from google.colab import auth
auth.authenticate_user()
# Importar biblioteca biqquery
from google.cloud import bigquery
# Bibliotecas Google Sheet
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe
# Montar Drive
from google.colab import drive
drive.mount('/content/drive')
# Modulos 
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content
!cp /content/drive/MyDrive/Colab_Notebooks/Autenticacoes/chaves_tokens.py /content
%run chaves_tokens.py
# Importar biblioteca biqquery
from modulos_colab import dados

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [ ]:
# Envio Parceiro Atual
df = dados.consulta_bigquery( projeto_gcp_1, """SELECT DISTINCT

                                                        DATETIME_ADD(CAST(o.created_at AS DATETIME), INTERVAL -3 HOUR) as data_captado,
                                                        concat("LU-",o.order_id) as order_id,
                                                        so.sub_order_id , 
                                                        so.seller_id , 
                                                        s.description as ultimo_status,
                                                        date(f.deadline_date) as data_prometida_entrega,
                                                        me.ShippingId as opcao_entrega,
                                                        case when entrega.data_entrega is null then 'N' else 'S' end as entrega_realizada,
                                                        entrega.data_entrega,
                                                        IF (((date(f.deadline_date) >= date(entrega.data_entrega))),'S','N') as entrega_no_prazo,
                                                        ss.Assistido,
                                                        ss.Mundo,
                                                        c.status as status_integra,
                                                        date(c.shipped_carrier_date) as data_expedicao
                                                                                             
                                                                                                            
                                                                FROM maga-bigdata.maestro.order as o
                                                                left join maga-bigdata.maestro.sub_order as so on so.order_uuid = o.uuid
                                                                left join maga-bigdata.maestro.sales_channel as sc on o.sales_channel_id = sc.id
                                                                left join maga-bigdata.maestro.customer as cus on o.customer_uuid = cus.uuid
                                                                left join maga-bigdata.maestro.status as s on so.current_status_id = s.id
                                                                left join maga-bigdata.maestro.shipping f ON so.uuid = f.sub_order_uuid
                                                                --left join maga-bigdata.magaluentregas.shipping as me on me.order_id = o.order_id and so.seller_id = me.seller
                                                                left join maga-bigdata.bi_adm.tb_safra_seller as ss on ss.Seller_MarketPlace_ID = so.seller_id
                                                                left join maga-bigdata.integra.order as a on a.idsite = o.basket_id
                                                                left join maga-bigdata.integra.order_package as me on me.OrderId = a.id
                                                                left join maga-bigdata.integra.business b on a.business_id = b.id and so.seller_id = b.SellerIdOpenApi
                                                                left join maga-bigdata.integra.order_status c ON a.id = c.order_id_fk and c.Business_Id = b.id
                                                                left join (SELECT cast(datetime_add(cast(min(date) AS datetime), interval -3 hour)as date) as data_entrega, sub_order_uuid
                                                                    FROM `maga-bigdata.maestro.event`
                                                                    where status_id  in (10,21)
                                                                    group by sub_order_uuid) as entrega on so.uuid = entrega.sub_order_uuid


                                                                where sc.organization_uuid = '4da25f48-4193-45de-b4ed-9b8c93b7f987'
                                                                    and if(lower(cus.email) like '%@c2f.com%',0,1) = 1
                                                                    and if(cus.email = 'treinamento@luizalabs.com',0,1) = 1
                                                                    --and DATETIME_ADD(CAST(f.deadline_date AS DATETIME), INTERVAL -3 HOUR) between '2022-04-01' and current_date()
                                                                    and EXTRACT(YEAR FROM date(f.deadline_date)) = 2022
                                                                    and EXTRACT(MONTH FROM date(f.deadline_date)) = 06
                                                                    and date(f.deadline_date) <= current_date()
                                                                    --and if(o.order_id = me.OrderId,0,1) = 0
                                                                    and me.TransporterId  is  null
                                                                    and so.seller_id <> 'magazineluiza'
                                                                    --and so.seller_id = 'sostudo' 
                                                                    --and s.description = 'Entregue'
                                                                    --and o.order_id = '1054770379408753'
                                                                    and if(s.description = 'Cancelado',0,1) = 1
                                                                    and c.status not in ('CANCELED')""")

# Ingestão BigQuery
df.to_gbq(destination_table='marketplace_analytics.envio_parceiro_magalu_junho2022',project_id=projeto_gcp_1, if_exists='replace')

1it [00:27, 27.92s/it]


In [ ]:
print('Envio parceiro magalu atualizado!')